# 동적 기본 인수를 지정하려면 None과 docstring을 사용하자

키워드 인수의 기본값으로 비정적(non-static) 타입을 사용해야 할 때도 있다.

이벤트 발생 시각까지 포함해 로깅 메시지를 출력한다고 가정하자.

In [4]:
from datetime import datetime
from time import sleep

def log(message, when=datetime.now()):
    print('%s: %s' % (when, message))
    
log('Hi there!')
sleep(0.1)
log('Hi again!')

2021-02-09 12:31:10.304560: Hi there!
2021-02-09 12:31:10.304560: Hi again!


`datetime.now`는 함수 정의 시 한 번만 실행되므로 timestamp가 동일하게 출력된다.  __기본 인수의 값은 모듈의 로드될 때 한번만 평가되며 프로그램이 시작할 때 일어난다.__

결과가 기대한 대로 나오게 하려면 기본값을 None으로 설정하고 docstring으로 실제 동작을 문서화하는 게 관례다. None이 나타나면 기본값을 할당한다.

In [6]:
def log(message, when=None):
    """Log a message with a timestamp
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """

    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))

In [8]:
log('Hi there!')
sleep(0.1)
log('Hi again!')

2021-02-09 12:40:59.827806: Hi there!
2021-02-09 12:40:59.928833: Hi again!


기본 인수 값으로 None을 사용하는 방법은 인수가 mutable할 때 중요하다.

JSON data로 encoding된 값을 로드하는 예제

In [12]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [13]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


foo와 bar의 기본 파라미터가 같기 때문에 발생하는 문제다. __즉, 이 둘은 같은 dictionary 객체다.__

In [14]:
assert foo is bar

키워드 인수의 기본값을 None으로 설정하고 함수의 docstring에 동작을 문서화해서 이 문제를 고친다.

In [20]:
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

In [21]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}
